# EXIF tag writer 

In [79]:
import os
from pathlib import Path
from joblib import delayed, Parallel
import glob
import tqdm
import numpy as np

In [ ]:
%config Completer.use_jedi = False

In [ ]:
outdir = Path(r'data\right\33576_Right')
exifpath = Path(r'exiftool\exiftool.exe')
tag = 'MACS_RGB_Right_33576'

#### no parallelization necessary 

In [ ]:
def write_exif(outdir, tag, exifpath):
    s = f'{exifpath} -overwrite_original -Model="{tag}" {outdir}'
    print(s)
    os.system(s)

In [ ]:
%time write_exif(outdir, tag, exifpath)

In [ ]:
exifpath.exists()

### Rasterio read image stats
* quick with parallel reading

In [ ]:
import rasterio

In [ ]:
flist = list(outdir.glob('*.tif'))

In [74]:
def get_stats(filepath):
    with rasterio.open(filepath) as src:
        d = src.read()
        return d.min(), d.mean(), d.max()

In [76]:
%time stats = Parallel(n_jobs=-1)(delayed(get_stats)(f) for f in flist)

Wall time: 4.76 s


## Edit mipps 

In [81]:
from xml.dom.minidom import parse, parseString

In [121]:
import xml.dom.minidom as minidom

In [122]:
minidom.Node.toxml()

xml.dom.minidom.Node

In [85]:
mippsfile = Path('mipps_scripts') / '33576_all_taps_linearstretch.mipps'

In [113]:
parseString("
<QVariantMap>
    <QVariantMap name="Filter">
        <bool name="enabled" value="true"/>
        <double name="max" value="30"/>
        <double name="min" value="0"/>
        <QString name="name" value="Linear-Stretch"/>
    </QVariantMap>
</QVariantMap>
")

SyntaxError: EOL while scanning string literal (<ipython-input-113-adbc6dbe110d>, line 1)

In [95]:
dom = parse(open(mippsfile)).documentElement

In [106]:
filters = dom.childNodes[1].childNodes[1].childNodes

In [108]:
f1 = filters[1]

dict_keys([])

#### Exiftool parallelization tests 

In [ ]:
def write_exif_parallel(outdir, tag, exifpath):
    flist = outdir.glob('*.tif')
    for f in flist:
        s = f'{exifpath} -overwrite_original -Model="{tag}" {outdir}'
        print(s)
        os.system(s)

In [ ]:
def write_exiffile(outfile, tag, exifpath):
    s = f'{exifpath} -overwrite_original -Model="{tag}" {outfile}'
    os.system(s)

def write_exif_parallel2(outdir, tag, exifpath):
    flist = outdir.glob('*.tif')
    Parallel()(delayed(write_exiffile)(f, tag, exifpath) for f in tqdm.tqdm_notebook(flist))

In [ ]:
%time write_exif_parallel(outdir, tag, exifpath)

In [ ]:
%time write_exif_parallel2(outdir, tag, exifpath)